In [12]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [13]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=25, random_state=0)

In [14]:
algo = SVD()
algo.fit(trainset)

In [15]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25
prediction 결과의 최초 5개 추출


[Prediction(uid='917', iid='50', r_ui=3.0, est=4.146849485448304, details={'was_impossible': False}),
 Prediction(uid='235', iid='318', r_ui=5.0, est=4.520400903576995, details={'was_impossible': False}),
 Prediction(uid='339', iid='12', r_ui=5.0, est=4.623942501872979, details={'was_impossible': False}),
 Prediction(uid='878', iid='755', r_ui=2.0, est=2.8745161748003083, details={'was_impossible': False}),
 Prediction(uid='618', iid='56', r_ui=4.0, est=3.294303958592111, details={'was_impossible': False})]

In [16]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('917', '50', 4.146849485448304),
 ('235', '318', 4.520400903576995),
 ('339', '12', 4.623942501872979)]

In [17]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [18]:
accuracy.rmse(predictions)

RMSE: 0.8854


0.8853765107670645

In [19]:
import pandas as pd

ratings = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings.csv')

ratings.to_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv', index=False, header=False)

In [20]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv', reader=reader)

In [21]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [22]:
import pandas as pd
from surprise import Reader, Dataset

In [23]:
ratings = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [24]:
from surprise.model_selection import cross_validate

In [25]:
ratings = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8725  0.8730  0.8683  0.8782  0.8774  0.8739  0.0036  
MAE (testset)     0.6694  0.6711  0.6686  0.6727  0.6728  0.6709  0.0017  
Fit time          3.72    3.80    3.77    3.73    3.73    3.75    0.03    
Test time         0.09    0.09    0.15    0.14    0.09    0.11    0.03    


{'test_rmse': array([0.8724655 , 0.87301274, 0.86827299, 0.87824457, 0.87735325]),
 'test_mae': array([0.66944805, 0.67111795, 0.66857655, 0.67274272, 0.67278268]),
 'fit_time': (3.716062545776367,
  3.8028037548065186,
  3.766930103302002,
  3.7270352840423584,
  3.732022285461426),
 'test_time': (0.0887596607208252,
  0.09275197982788086,
  0.14563512802124023,
  0.14059901237487793,
  0.09377312660217285)}

In [26]:
from surprise.model_selection import GridSearchCV

In [27]:
param_grid = {'n_epochs' : [20, 40, 60], 'n_factors' : [50, 100, 200]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8772360264216198
{'n_epochs': 20, 'n_factors': 50}


In [28]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [31]:
from surprise.dataset import DatasetAutoFolds

In [32]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

data_folds = DatasetAutoFolds(ratings_file = r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv',
                             reader=reader)

trainset = data_folds.build_full_trainset()

In [33]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [34]:
movies = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv')

movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음


KeyError: 'movieId'

In [36]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [37]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    total_movies = movies['movieId'].tolist()
    
    unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:',
         len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

KeyError: 'movieId'

In [38]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids,
                                                                         top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print(' #### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

KeyError: 'movieId'